In [1]:
import json
import os
from pprint import pprint
from tqdm import tqdm

# PATH_TO_DATA = 'data/processed_for_chroma/reviews'
# FILENAMES = os.listdir(PATH_TO_DATA)
# review_data = dict()

# for filename in FILENAMES:
#     with open(f'{PATH_TO_DATA}/{filename}', 'r') as file:
#         review_data[os.path.splitext(os.path.basename(filename))[0]] = json.load(file)

# # This one is missing the doi key
# del review_data['Earth_Science_Reviews']['metadatas'][292]
# del review_data['Earth_Science_Reviews']['documents'][292]
# del review_data['Earth_Science_Reviews']['ids'][292]

# # postprocessing
# for journal in review_data:
#     for i, d in enumerate(review_data[journal]['metadatas']):
#         # Convert stringified list to list
#         d['reference'] = json.loads(d['reference'])
#         d['doi'] = json.loads(d['doi'])

# print(review_data.keys())
# print(f"Journal keys: {review_data['Astro_Reviews'].keys()}")
# paper = review_data['Astro_Reviews']['metadatas'][0]
# print(paper.keys())
# print(paper['doi'])
# print(type(paper['doi']))
# print(paper['reference'][:3])
# all_reviews = [
#     record for journal in review_data for record in review_data[journal]['metadatas']]
# print(len(all_reviews))
# print(all_reviews[0].keys())

In [2]:
from utils import load_dataset

PATH_TO_DATA = 'data/json/Astro_Reviews.json'
astro_reviews = load_dataset(PATH_TO_DATA)
print(astro_reviews[0].keys())

astro_research = load_dataset('data/json/Astro_Research.json')

all_records = astro_reviews + astro_research

data/json/Astro_Reviews.json: 996/1000 have all required keys
dict_keys(['bibcode', 'abstract', 'aff', 'author', 'bibstem', 'doctype', 'doi', 'id', 'keyword', 'pubdate', 'title', 'read_count', 'reference', 'citation_count', 'citation', 'body'])
data/json/Astro_Research.json: 981/1000 have all required keys


In [3]:
# PATH_TO_DATA = 'data/processed_for_chroma/research'
# FILENAMES = os.listdir(PATH_TO_DATA)
# research_data = dict()

# for filename in FILENAMES:
#     with open(f'{PATH_TO_DATA}/{filename}', 'r') as file:
#         research_data[os.path.splitext(os.path.basename(filename))[
#             0]] = json.load(file)


# # postprocessing
# for journal in research_data:
#     for i, d in enumerate(research_data[journal]['metadatas']):
#         if not 'doi' in d:
#             del research_data[journal]['metadatas'][i]
#             del research_data[journal]['documents'][i]
#             del research_data[journal]['ids'][i]

# for journal in research_data:
#     for d in research_data[journal]['metadatas']:

#         # Convert stringified list to list
#         d['reference'] = json.loads(d['reference'])
#         d['doi'] = json.loads(d['doi'])

# print(research_data.keys())
# research_data['Astro_Research'].keys()

# all_data = all_reviews + \
#     [record for journal in review_data for record in review_data[journal]['metadatas']]
# print(f"All data: {len(all_data)}")

In [4]:

import re

# Define bibcode and citation patterns
lastname = r"[A-Z][a-zA-ZÀ-ÖØ-öø-ÿ-]*(?:'[A-Z][a-zA-ZÀ-ÖØ-öø-ÿ-]*)?"
year = r"\(?(\d{4}[a-z]?)\)?"
name_sep = r",?\s"
INLINE_CITATION_PATTERN = fr"({lastname}(?:{name_sep}{lastname})*(?: et al.?)?)\s*{year}"

# Compile the regex pattern
inline_regex = re.compile(INLINE_CITATION_PATTERN)

test = " Delbouille et al. 1981 the future (Section 5). 2. INGREDIENTS FOR SOLAR ABUNDANCE ANALYSIS 2.1. Observations Analyses of th"

# Find all matches using the compiled pattern
matches = inline_regex.finditer(test)
results = [match for match in matches]
print(f"Results: {results}")


Results: [<re.Match object; span=(1, 23), match='Delbouille et al. 1981'>]


### Functions to resolve incline citation to bibcode

In [5]:
# for each result, get the first letter of the first author's last name
# get the year
def bibcode_regex(author: str, year: str):
    """
    Given first author and year, return a regex pattern for the
    corresponding bibcode
    """
    initial = author[0]
    year = year[:4] # cut off any letters at the end
    pattern = fr'^{year}.*{initial}$'
    return re.compile(pattern)

def bibcode_matches(inline_citation: tuple[str, str], references: list[str]) -> int:
    """
    Given an inline citation and a list of references, return the number of
    references that match the inline citation's bibcode regex pattern
    """
    pattern = bibcode_regex(*inline_citation)
    return [s for s in references if pattern.match(s)]

# def make_citation_bibcode_list(inline_citations: list[tuple[str, str]], references: list[str]) -> list[tuple[tuple[str, str], str]]:
#     """
#     Given a paper's list of inline citations and list of references, return a list of
#     tuples where the first element is the inline citation and the second element
#     is the corresponding bibcode from the references list where there is exactly one match
#     """
#     return [(citation, matches[0]) for citation in inline_citations 
#             if len((matches := bibcode_matches(citation, references))) == 1]

# usable_citations = make_citation_bibcode_list(inline_citations, paper['reference'])
# print(f"Results: {len(usable_citations)}")
# print(usable_citations[:5])

In [6]:
import pysbd

def get_inline_citations(text: str) -> list[tuple[str, str]]:
    return [match.groups() for match in inline_regex.finditer(text)]

def sentence_to_example(record, sentence, all_records):
    """
    Takes all the inline citations of a sentence and if it can resolve them to dois
    then it returns the """
    def citation_to_doi(citation):
        """
        Takes a single inline citation as tuple of (author, year) and determines if there is a unique
        matching bibcode in the record's references. If so, it continues to look for a unique
        doi matching that bibcode in the entire dataset. It returns the doi if resolved, otherwise None.
        """
        bibcodes = bibcode_matches(citation, record['reference'])
        if len(bibcodes) != 1:
            return None
        
        matching_dois = [record['doi'][0]
                         for record in all_records if record['bibcode'] == bibcodes[0]]
        if len(matching_dois) != 1:
            return None
        return matching_dois[0]
    
    inline_citations = get_inline_citations(sentence)
    citation_dois = []
    for citation in inline_citations:
        if not (doi := citation_to_doi(citation)):
            break
        citation_dois.append(doi)

    # If all citations resolved to dois, return the example
    if len(inline_citations) != len(citation_dois):
        return None
    return {
            'source_doi': record['doi'][0],
            'sentence': sentence,
            'citation_dois': citation_dois
           }


def create_examples_from_record(record):
    splitter = pysbd.Segmenter(language="en", clean=False)
    sentences = [s for s in splitter.segment(record['body']) if len(s) > 40]
    return [
        example for sentence in sentences if (example := sentence_to_example(record, sentence))
    ]
        

In [7]:
def find_salvageable_bibcodes(record, sentence, all_bibcodes):
    """
    Similar logic to sentence_to_example, except this function, when it finds an inline citation
    that resolves to a single bibcode in a record/sentence, but it CANNOT find the corresponding
    doi in the entire dataset (i.e. the bibcode is not in the dataset), it returns the bibcode
    """
    def citation_to_missing_bibcode(citation):
        """
        Takes a single inline citation as tuple of (author, year) and determines if there is a unique
        matching bibcode in the record's references. If so, it continues to look for a unique
        doi matching that bibcode in the entire dataset. It returns the doi if resolved, otherwise None.
        """
        bibcodes = bibcode_matches(citation, record['reference'])
        if len(bibcodes) != 1:
            return None
        
        bibcode = bibcodes[0]
        
        # The inline citation corresponds to a unique bibcode, but that bibcode is not in the dataset
        if bibcode not in all_bibcodes:
            return bibcode
        
        matching_dois = [record['doi'][0]
                         for record in all_records if record['bibcode'] == bibcode]
        if len(matching_dois) != 1:
            return None
        return matching_dois[0]

    inline_citations = get_inline_citations(sentence)
    return [bibcode for citation in inline_citations if (bibcode := citation_to_missing_bibcode(citation))]


In [8]:
all_bibcodes = [record['bibcode'] for record in all_records]
len(all_bibcodes) == len(all_records)

True

In [9]:
import pysbd

salvageable_bibcodes = set()
splitter = pysbd.Segmenter(language="en", clean=False)
for record in tqdm(astro_reviews):
    sentences = [s for s in splitter.segment(record['body']) if len(s) > 40]
    for sentence in sentences:
        salvageable_bibcodes.update(find_salvageable_bibcodes(record, sentence, all_bibcodes))

print(f"Salvageable bibcodes: {len(salvageable_bibcodes)}")

100%|██████████| 996/996 [1:06:51<00:00,  4.03s/it]

Salvageable bibcodes: 46333


In [15]:
earth_reviews = load_dataset('data/json/Earth_Science_Reviews.json')
earth_research = load_dataset('data/json/Earth_Science_Research.json')
planet_reviews = load_dataset('data/json/Planetary_Reviews.json')
planet_research = load_dataset('data/json/Planetary_Research.json')

all_records = astro_reviews + astro_research + earth_reviews + earth_research + planet_reviews + planet_research
all_bibcodes = [record['bibcode'] for record in all_records]

data/json/Earth_Science_Reviews.json: 994/1000 have all required keys
data/json/Earth_Science_Research.json: 1000/1000 have all required keys
data/json/Planetary_Reviews.json: 994/1000 have all required keys
data/json/Planetary_Research.json: 964/1000 have all required keys


In [16]:
for record in tqdm(earth_reviews + planet_reviews):
    sentences = [s for s in splitter.segment(record['body']) if len(s) > 40]
    for sentence in sentences:
        salvageable_bibcodes.update(find_salvageable_bibcodes(record, sentence, all_bibcodes))

print(f"Salvageable bibcodes: {len(salvageable_bibcodes)}")

100%|██████████| 1988/1988 [2:44:51<00:00,  4.98s/it]   

Salvageable bibcodes: 74365


In [17]:
with open('data/salvageable_bibcodes.txt', 'w') as file:
    for bibcode in salvageable_bibcodes:
        file.write(f"{bibcode}\n")

In [16]:
with open('data/test_set.jsonl', 'a') as file:
    for journal in review_data:
        for record in review_data[journal]['metadatas'][:10]:
            examples = create_examples_from_record(record)
            for example in examples:
                json.dump(example, file)
                file.write('\n')

In [ ]:
num_sentences = []
num_examples = []
examples_with_empty_labels = []
total_examples = 0
for i, record in tqdm(enumerate(review_data['Astro_Reviews']['metadatas'][:100])):
    splitter = pysbd.Segmenter(language="en", clean=False)
    sentences = [s for s in splitter.segment(record['body']) if len(s) > 40]
    num_sentences.append(len(sentences))
    examples = [example for sentence in sentences if (example := sentence_to_example(record, sentence))]
    num_examples.append(len(examples))
    examples_with_empty_labels.append(len([e for e in examples if e['citation_dois'] == []]))

In [ ]:
print(f"Total number of examples: {sum(num_examples)}")
print(f"Total number of examples with no citations: {sum(examples_with_empty_labels)}")

In [ ]:
num_examples

In [13]:
paper = astro_reviews[0]
paper['body']
paper['reference']

['1929ApJ....70...11R',
 '1956RvMP...28...53S',
 '1958ZA.....46..108B',
 '1960ApJS....5....1G',
 '1962ApJ...136..906V',
 '1967ZA.....65..365H',
 '1968ZPhy..211..404D',
 '1969ApL.....4..143H',
 '1972ApJ...175L..95H',
 '1972SoPh...26..250L',
 '1973apds.book.....D',
 '1974SoPh...39...19H',
 '1975A&A....42...37C',
 '1975SoPh...41...53M',
 '1976ApJS...30....1V',
 '1976Sci...191.1223R',
 '1978MNRAS.182..249L',
 '1978MNRAS.183...79L',
 '1982A&A...107....1N',
 '1982SoPh...81....3H',
 '1984A&A...141...10G',
 '1984ApJ...282..330S',
 '1984SoPh...90..205N',
 '1984sfat.book.....K',
 '1986A&A...164..395Y',
 '1988A&A...203..378Y',
 '1988ApJ...331..815M',
 '1989GeCoA..53..197A',
 '1989hra2.book.....F',
 '1990A&A...232..225G',
 '1990A&A...239..367Y',
 '1991A&A...242..488G',
 '1992A&A...259..301H',
 '1992RMxAA..23...45K',
 '1993A&A...275..101E',
 '1993A&A...275..269K',
 '1993ASPC...44..496C',
 '1993oee..conf...15G',
 '1994RSPTA.349..213G',
 '1994SoPh..153...91N',
 '1995A&A...296..217B',
 '1995A&A...296.